# Aperture and PSF photometry

[Photutils](https://photutils.readthedocs.io/en/stable/) is an Astropy coordinated package that provides tools for detecting and performing photometry of astronomical sources.


<section class="objectives panel panel-warning">
<div class="panel-heading">
<h2><span class="fa fa-certificate"></span> Objectives</h2>
</div>


<div class="panel-body">

<ul>
<li>Estimate and subtract backgrounds in images</li>
<li>Detect sources</li>
<li>Do aperture photometry</li>
<li>Construct a PSF</li>
<li>Do PSF photometry</li>
</ul>

</div>

</section>


## Documentation

This notebook only shows a subset of the functionality in photutils. For more information about the features presented below as well as other available features, you can read the
[photutils documentation](https://photutils.readthedocs.io/en/stable/).

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rc('image', origin='lower')
plt.rc('figure', figsize=(10, 6))

## Background estimation and subtraction

Given a calibrated image, your first step (depending on the data) might be to subtract the background. We start off by loading one of the example datasets from photutils:

In [ ]:
from photutils.datasets import load_star_image
star_image = load_star_image()
star_image

The ``star_image`` object is an HDU, so we need to access the data array when plotting it:

In [ ]:
plt.imshow(star_image.data)

We can now make a histogram to check if the image has a significant background:

In [ ]:
_ = plt.hist(star_image.data.ravel(), bins=100)

It looks like there is a background! There are many different ways of correcting for this, and one of the simplest ways would be for example to sigma clip the image and use the median of the remaining pixel values. But in some cases a more sophisticated approach is needed, so we can try and estimate the background assuming that it may be spatially varying:

In [ ]:
from astropy.stats import SigmaClip
from photutils import Background2D, MedianBackground

In [ ]:
sigma_clip = SigmaClip(sigma=3.)
bkg_estimator = MedianBackground()

In [ ]:
bkg = Background2D(star_image.data, (265, 265),
                   sigma_clip=sigma_clip,
                   bkg_estimator=bkg_estimator)

In [ ]:
plt.imshow(bkg.background)

In [ ]:
star_image_nobkg = star_image.data - bkg.background

In [ ]:
plt.imshow(star_image_nobkg, vmin=-1000, vmax=2000)

In [ ]:
_ = plt.hist(star_image_nobkg.ravel(), bins=100)

## Source detection

Photutils includes several algorithms for detecting sources, including for example an implementation of the [DAOFIND](https://iraf.net/irafhelp.php?val=daofind) algorithm. First, we will need an estimate of the background noise, which we can get with sigma clipping:

In [ ]:
from astropy.stats import sigma_clipped_stats
mean, median, std = sigma_clipped_stats(star_image_nobkg, sigma=3.0)
print(mean, median, std)

We next run the source detection:

In [ ]:
from photutils import DAOStarFinder
daofind = DAOStarFinder(fwhm=5.0, threshold=10.*std) 
sources = daofind(star_image_nobkg) 

The result is an astropy Table:

In [ ]:
sources

In [ ]:
subset = star_image_nobkg.copy()
subset[subset < 30 * std] = 0

In [ ]:
plt.imshow(star_image_nobkg)
plt.plot(sources['xcentroid'], sources['ycentroid'], 'o', mfc='none', mec='white')
plt.xlim(200, 600)
plt.ylim(200, 600)

## Aperture photometry

Now that we have the position of sources, we can perform aperture photometry. The first step is to create an aperture object whcih contains all the positions for which we want to measure the fluxes:

In [ ]:
from photutils import CircularAperture
apertures = CircularAperture(list(zip(sources['xcentroid'], sources['ycentroid'])), r=3.)

For simplicity we assume apertures of constant radii (3 pixels). We can now measure the fluxes:

In [ ]:
from photutils import aperture_photometry
apphot_table = aperture_photometry(star_image_nobkg, apertures)
apphot_table

In [ ]:
plt.scatter(apphot_table['xcenter'], apphot_table['ycenter'], s=apphot_table['aperture_sum'] / 10000)

Note that photutils also includes functionality for PSF photometry but this is more advanced, so we do not look at it here.


<section class="challenge panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Challenge</h2>
</div>


<div class="panel-body">

<ol>
<li>Carry out aperture photometry using a radius of 5 and 10 pixels, and compare the results in a scatter plot</li>
<li>Using this, construct a table containing just the sources where the 5 and 10 pixel fluxes agree to within a factor of 2</li>
</ol>

</div>

</section>


In [ ]:
#1
apertures_5 = CircularAperture(list(zip(sources['xcentroid'], sources['ycentroid'])), r=5.)
apphot_table_5 = aperture_photometry(star_image_nobkg, apertures_5)
apertures_10 = CircularAperture(list(zip(sources['xcentroid'], sources['ycentroid'])), r=10.)
apphot_table_10 = aperture_photometry(star_image_nobkg, apertures_10)
plt.loglog(apphot_table_5['aperture_sum'], apphot_table_10['aperture_sum'], '.')
plt.plot([1e3, 1e7], [1e3, 1e7])

In [ ]:
#2
frac = apphot_table_5['aperture_sum'] / apphot_table_10['aperture_sum']
agree = (frac < 2) & (frac > 0.5)
subtable = sources[agree]
subtable

<center><i>This notebook was written by <a href="https://aperiosoftware.com/">Aperio Software Ltd.</a> &copy; 2019, and is licensed under a <a href="https://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License (CC BY 4.0)</a></i></center>

![cc](https://mirrors.creativecommons.org/presskit/buttons/88x31/svg/by.svg)